In [1]:
import cv2, sys, os, concurrent.futures, numpy as np
from sklearn.ensemble import IsolationForest
from skimage import filters
from scipy.spatial import distance
from intersections import find_intersections_via_hit_or_miss

input_dir = "/home/neutral/Documents/Wings/modified_wings_labeled"
clean_dir = "/home/neutral/Documents/Wings/Mod-labeled-Clean"
noisy_dir = "/home/neutral/Documents/Wings/Mod-labeled-Noisy"

os.makedirs(clean_dir, exist_ok=True)
os.makedirs(noisy_dir, exist_ok=True)

counter = 0

def load_images():
    global input_dir, clean_dir, noisy_dir

    # Read every image
    images = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    images.sort()

    return images

def progress_bar(len, counter, txt="Loading:"):
    percentage = int(counter * 100 / len)
    print(f"{txt} |{'='*percentage}{'-'*(100-percentage)}| {percentage}%", end='\r')
    if counter == len:
        sys.stdout.write(f"\r{txt} 100%\033[K\n")

In [2]:
def data_generator():
    data = list()
    images = load_images()

    print("Extracting image intersections:")

    # Helper function to return a tuple that contains the image name and its intersections
    def single_image_intersections(image_path):
        global counter
        # image_path = os.path.join(input_dir, image_path)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        intersections = find_intersections_via_hit_or_miss(image)
        counter += 1

        # Simple console animation to make the waiting time entertaining :D
        progress_bar(len(images), counter)

        return (os.path.basename(image_path), intersections)
    
    # Parallel intersection extraction
    def parallel_image_processing():
        with concurrent.futures.ThreadPoolExecutor() as executor:
            result = list(executor.map(single_image_intersections, images))
        return result
    
    return parallel_image_processing()

In [3]:
def min_distance(coord, other_object):
    return min([distance.euclidean(coord, other_coord) for other_coord in other_object])

# Function to find the closest matches of coordinates between two objects
def compare_objects(object1, object2, threshold=1.0):
    matched_coords = 0
    for coord in object1:
        if min_distance(coord, object2) < threshold:
            matched_coords += 1
    return matched_coords

# Function to identify and exclude odd objects
def exclude_odd_objects(objects_list, match_threshold=0.8, distance_threshold=10):
    valid_objects = []
    for obj in objects_list:
        match_count = 0
        for other_obj in objects_list:
            if obj is not other_obj:
                # Compare each object with all others
                matched = compare_objects(obj, other_obj, distance_threshold)
                if matched / len(obj) >= match_threshold:
                    match_count += 1
        # Exclude object if not enough matches found
        if match_count >= len(objects_list) * match_threshold:
            valid_objects.append(obj)
    return valid_objects

In [4]:
data = data_generator()
data = [obj[1] for obj in data]

Extracting image intersections:
Loading: 100%==============================================================================================| 100%oading: |===================================================================================================-| 99%


In [5]:
print(exclude_odd_objects(data))

ValueError: min() iterable argument is empty